In [4]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [5]:
# get data
data = pd.read_csv('Level_3c_NonPath_SP3_PsPu_PSL_1_2_3_435.csv') 

# split data
X=data.iloc[:,1:51]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A      Xc1.R     Xc1.N  ...  Xc2.lambda28  Xc2.lambda29  Xc2.lambda30
0    17.569047  10.980654  2.440145  ...      0.025292      0.026712      0.026187
1     7.805162   5.676482  2.365201  ...      0.026514      0.026683      0.025076
2     9.759306   7.617019  4.284573  ...      0.026153      0.025768      0.024628
3    13.897496   6.434026  2.573610  ...      0.024729      0.024340      0.026954
4    13.083343   6.413403  2.821897  ...      0.024980      0.024514      0.026914
..         ...        ...       ...  ...           ...           ...           ...
430   3.434023   2.113245  1.849089  ...      0.024686      0.022914      0.025301
431   8.597566   6.513308  3.647452  ...      0.025071      0.024687      0.022506
432   8.597410   1.719482  3.152384  ...      0.024239      0.024058      0.021686
433   5.471856   0.521129  1.563388  ...      0.022357      0.026263      0.022232
434   5.471856   0.521129  1.563388  ...      0.022357      0.026263      0.022232

[43

In [6]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [7]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(50, 3 - 1) = 2 components.
  ChangedBehaviorWarni

In [8]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(3 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 128)            91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 151,651
Trainable params: 151,651
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
21/21 [==============================] - 39s 169ms/step - loss: 0.6839 - accuracy: 0.6069 - val_loss: 0.6330 - val_accuracy: 0.5569
Epoch 2/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.6086 - accuracy: 0.5505 - val_loss: 0.4925 - val_accuracy: 0.5689
Epoch 3/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.4945 - accuracy: 0.6305 - val_loss: 0.3671 - val_accuracy: 0.8743
Epoch 4/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.4369 - accuracy: 0.6991 - val_loss: 0.2582 - val_accuracy: 0.8563
Epoch 5/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.3876 - accuracy: 0.6777 - val_loss: 0.2262 - val_accuracy: 0.8683
Epoch 6/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.3301 - accuracy: 0.7608 - val_loss: 0.1921 - val_accuracy: 0.8922
Epoch 7/1000
21/21 [==============================] - 0s 19ms/step - loss: 0.3209 - accuracy: 0.7583 - val_loss: 0.1929 - val_accuracy: 0.88

In [11]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.9250374812593704
test-acc = 0.9401197604790419
train-CFR =               precision    recall  f1-score   support

           0       0.89      0.92      0.90       220
           1       0.93      0.89      0.91       230
           2       0.96      0.97      0.96       217

    accuracy                           0.93       667
   macro avg       0.93      0.93      0.93       667
weighted avg       0.93      0.93      0.92       667

test-CFR =               precision    recall  f1-score   support

           0       0.90      0.97      0.93        58
           1       0.93      0.88      0.90        48
           2       0.98      0.97      0.98        61

    accuracy                           0.94       167
   macro avg       0.94      0.94      0.94       167
weighted avg       0.94      0.94      0.94       167

